In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from hmmlearn.hmm import CategoricalHMM
from sklearn.metrics import log_loss, roc_auc_score

In [2]:
data = pd.read_csv("../Data/sample.csv")

In [3]:
data = data.sort_values(by=['user_xid', 'skill_id', 'start_time'])

In [4]:
grouped = data.groupby(by=['skill_id', 'user_xid'])
grouped.head()

,problem_log_id,user_xid,old_problem_id,skill_id,start_time,discrete_score,attempt_count,hint_count,saw_answer
7419762,183179369,20,1507769,237,2024-01-12 10:46:49.603 -0500,1,1,0,False
7419910,183179918,20,1842305,237,2024-01-12 10:51:02.261 -0500,0,2,0,False
7419562,183180043,20,1842306,237,2024-01-12 10:51:57.322 -0500,1,1,0,False
2573312,190454857,20,2096006,1126,2024-05-21 09:30:43.017 -0400,1,1,0,False
2573437,190454862,20,2095985,1126,2024-05-21 09:30:50.859 -0400,0,4,0,True
...,...,...,...,...,...,...,...,...,...
2357043,190621531,4106992,1805182,372,2024-05-31 15:54:32.828 -0400,1,1,0,False
2357044,190621531,4106992,1805182,372,2024-05-31 15:54:32.828 -0400,1,1,0,False
2357045,190621531,4106992,1805182,372,2024-05-31 15:54:32.828 -0400,1,1,0,False
2357046,190621531,4106992,1805182,372,2024-05-31 15:54:32.828 -0400,1,1,0,False


In [6]:
# f1 = open("group.txt", "w")

curr_skill = 1
skill_dict = {}
value_dict = {"seq": [], "lengths": []}

# count = 0
for value, group in tqdm(grouped):
    # if count == 100:
    #     break    
    # border = '===============================================\n'
    # group_string = str(group['discrete_score'])
    # write_value = border + 'skill_id: ' + str(value[0]) + '\n' + 'user_xid: ' + str(value[1]) + '\n' + group_string + '\n\n'
    # f1.write(write_value)
    
    if value[0] != curr_skill:
        value_dict['seq'] = np.concatenate(value_dict['seq']).reshape(-1,1)
        skill_dict[curr_skill] = value_dict
        value_dict = {"seq": [], "lengths": []}
    
    curr_skill = value[0]
    value_dict['seq'].append(group.discrete_score.to_numpy())
    value_dict['lengths'].append(group.shape[0])
    # count += 1
value_dict['seq'] = np.concatenate(value_dict['seq']).reshape(-1,1).astype(int)
skill_dict[curr_skill] = value_dict
    
# f1.close()
    

100%|██████████| 1424943/1424943 [00:48<00:00, 29227.00it/s]


In [7]:
start_prob = np.array([0.5, 0.5])
trans_prob = np.array([[0.7, 0.3],[0.3, 0.7]])
emission_prob = np.array([[0.8, 0.2],[0.2, 0.8]])

model = CategoricalHMM(n_components=2, init_params="", params="st", startprob_prior=start_prob, transmat_prior=trans_prob, emissionprob_prior=emission_prob, n_iter=100, verbose=True)

In [9]:
count = 0
for skill in skill_dict:
    if count > 10:
        break    
        
    X = skill_dict[skill]['seq']
    lengths = skill_dict[skill]['lengths']
    
    model.fit(X, lengths)
    
    print("Finished model training. Printing final statistics...")

    # print("Finished model training. Printing final statistics...")
    # y_score = model.predict_proba(X)[:, 1]
    # 
    # loss = log_loss(X, y_score)     # logistical loss
    # auc = roc_auc_score(X, y_score)
    # 
    # print(f"Loss: {loss}")
    # print(f"AUC:  {auc}")
    
    count += 1

         1  -53607.64397657             +nan
         2  -51991.86819913   +1615.77577744
         3  -51991.86819913      +0.00000000
         1  -12121.62174875             +nan
         2  -12063.86109050     +57.76065825


Finished model training. Printing final statistics...


         3  -12063.86109050      -0.00000000
         1   -5140.59507372             +nan
         2   -5247.18695019    -106.59187647
Model is not converging.  Current: -5247.186950194663 is not greater than -5140.595073724729. Delta is -106.5918764699336


Finished model training. Printing final statistics...
Finished model training. Printing final statistics...


         1  -28795.91864888             +nan
         2  -28005.93659953    +789.98204935
         3  -28005.93659953      +0.00000000
         1 -120700.03708442             +nan


...

Finished model training. Printing final statistics...
Finished model training. Printing final statistics...
Finished model training. Printing final statistics...


         1  -17870.16839530             +nan
         2  -17801.26634468     +68.90205062
         3  -17801.26634468      +0.00000000
         1  -65724.30102213             +nan


Finished model training. Printing final statistics...


         2  -66792.08376627   -1067.78274414
Model is not converging.  Current: -66792.08376627264 is not greater than -65724.30102213449. Delta is -1067.7827441381523


Finished model training. Printing final statistics...
